# Visualisations of Bayesian predictions
<br>

## execute: Cell -> Run all

In [ ]:
import numpy as np
import pandas as pd

from scipy.stats import gaussian_kde
import arviz as az

from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import ColumnDataSource, LinearAxis, Range1d, HoverTool, Legend, Span, Text, Title, Circle, Label
from bokeh.io import output_notebook, show
output_notebook()

import warnings
warnings.filterwarnings('ignore')

In [ ]:
full_traces= np.load('traces.npy')

full_index=pd.read_csv('db.csv')

mask_ = np.mean(full_traces, axis=1)>0.1

In [ ]:
def plot_pdf(vis_index=None, traces=full_traces, threshold_val=0.1, title='Trace pdf', colored_patches=False, x_range_=[-0.1,0.6], smoothness=2000):
    '''
    defaults:
    plot_pdf(vis_index=None, traces=None, threshold_val=0.1, title='Trace pdf', colored_patches=False, x_range_=[-0.1,0.6], smoothness=2000)
    '''
    trace= traces[vis_index]
    dens= gaussian_kde(trace)
    x_dim=np.linspace(-1.0,1.5,smoothness)
    dens._compute_covariance()
    
    source = ColumnDataSource(data=dict(
        mean =[np.mean(trace)],
        y = [0]))
    
    tooltip = [("mean", "@mean")]
    p = figure(plot_width=400, plot_height=400, title=title, x_range=(x_range_[0],x_range_[1]), tooltips=tooltip)
    if colored_patches and source.data['mean'][0]>=threshold_val:
        p.patch(x_dim, dens(x_dim), line_width=2, line_color="#ff5300", color="#ff5300",  alpha=0.6)
    else:
        p.patch(x_dim, dens(x_dim), line_width=2, line_color="#006fa2", color="#006fa2",  alpha=0.6)
    threshold = Span(location=threshold_val,
                                  dimension='height', line_color='#ff5300',
                                  line_dash='dashed', line_width=2, name='threshold', line_alpha=0.6)
    p.add_layout(threshold)
    
    p.circle('mean', 'y', fill_color="#2166ac", line_color="#2166ac", size=4, fill_alpha=0.6, source=source)
    
#     p.yaxis.major_tick_line_color = None
#     p.yaxis.minor_tick_line_color = None
    p.yaxis.major_label_text_font_size = '0pt'
    p.yaxis.major_label_text_color = None
    p.axis.minor_tick_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.axis_line_color = None
    p.background_fill_color = "#fafafa"
    show(p)
    

In [ ]:
def plot_pdfs_sorted(traces, threshold_val=0.1, display_under_threshold=True, sorted_means=True, ascending=True, title='posterior traces',
                     colored_patches=False, x_range_=[-0.1,0.6], smoothness=200,jd_mode=False, size=[800, 1500], line_color="#006fa2",
                     line_color_match="#ff5300", y_range_add=2):
    '''
    defaults:
    plot_pdfs_sorted(traces, threshold_val=0.1, display_under_threshold=True, sorted_means=True, ascending=True, title='posterior traces',
                     colored_patches=False, x_range_=[-0.1,0.6], smoothness=200,jd_mode=False, size=[800, 1500], line_color="#006fa2",
                     line_color_match="#ff5300", y_range_add=2)
    '''
    trace_means= np.mean(traces, axis=1)    
    x_dim=np.linspace(-1.0, 1.5, smoothness)
    index=np.arange(0, traces.shape[0])
    db_index= full_index['index'].values
    db= full_index['db'].values
    
    if not display_under_threshold:
        mask= trace_means>0.1
        trace_means= trace_means[mask]
        index=index[mask]
        db_index= db_index[mask]
        db= db[mask]
        traces= traces[mask]
    
    order=np.arange(0, trace_means.shape[0])
    if sorted_means and ascending:
        order= np.argsort(trace_means)[::-1]
    if sorted_means and not ascending:
        order= np.argsort(trace_means)

    source = ColumnDataSource(data=dict(
        target_means = trace_means[order],
        display_index= index[order],
        db_index= db_index[order],
        database= db[order],
        index_ =range(trace_means.shape[0])))

    p = figure(plot_width=size[0], plot_height=size[1], title=title, y_range=(0-y_range_add, trace_means.shape[0]+y_range_add), x_range=(x_range_[0],x_range_[1]))

    for i, order in zip(order, range(traces.shape[0])):
        dens= gaussian_kde(traces[i])
        dens._compute_covariance()
        if colored_patches:
            if trace_means[i]>=threshold_val:
                p.patch(x_dim, dens(x_dim)+ order, line_width=1, line_color=line_color_match, color="#ff5300", alpha=0.6)
                p.line(x_dim, dens(x_dim)+ order, line_width=1, line_color="black", alpha=0.0)
            else:
                p.patch(x_dim, dens(x_dim)+ order, line_width=1, line_color=line_color, color="#006fa2", alpha=0.6)
                p.line(x_dim, dens(x_dim)+ order, line_width=1, line_color="black", alpha=0.0)
        else:
            if jd_mode:
                p.patch(x_dim, dens(x_dim)+ order, line_width=2, line_color="white", color=None, alpha=0.6)
            else:
                p.patch(x_dim, dens(x_dim)+ order, line_width=1, line_color=line_color, color="#006fa2", alpha=0.6)
                p.line(x_dim, dens(x_dim)+ order, line_width=1, line_color="black", alpha=0.0)
                
    
    threshold = Span(location=threshold_val,
                                  dimension='height', line_color='#ff5300',
                                  line_dash='dashed', line_width=2, name='threshold', line_alpha=0.6)
    if not jd_mode:
        circles= Circle(x='target_means', y='index_', fill_color="#2166ac", line_color="#2166ac", size=4, fill_alpha=0.6) #, source=source)
        circles_r = p.add_glyph(source_or_glyph=source, glyph=circles)
        circles_hover = HoverTool(renderers=[circles_r], tooltips = [("vis_index", "@display_index"), ("excel index", "@db_index"), ("database", "@database")])
        p.add_tools(circles_hover)
        p.add_layout(threshold)

    p.outline_line_color = None
    p.background_fill_color = "#efefef"
    if jd_mode:
        p.background_fill_color = "black"
        p.title.text_color= None
        p.add_layout(Title(text="UNKNOWN DISTRIBUTIONS", align="center", text_color="white",text_font_size='20pt'), "below")
        p.yaxis.major_label_text_color = None
        p.xaxis.major_label_text_color = None
        p.toolbar_location=None
        p.border_fill_color = "black"
        p.min_border_left = 80
        p.min_border_right = 80
        p.min_border_top = 80
        p.min_border_bottom = 110

    p.ygrid.grid_line_color = None
    p.xgrid.grid_line_color = "#dddddd"
#     p.xgrid.ticker = p.xaxis.ticker

    p.axis.minor_tick_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.axis_line_color = None
    
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    
    show(p)

# Posterior traces of all datapoints

In [ ]:
plot_pdfs_sorted(full_traces, threshold_val=0.1, sorted_means=True, ascending=True, colored_patches=False, x_range_=[-0.1,1],
                 title='posterior traces', jd_mode=False, line_color=None, line_color_match=None, y_range_add=10)

# Posterior traces of actives (only Bayesian model)

In [ ]:
plot_pdfs_sorted(full_traces, threshold_val=0.1, display_under_threshold=False, sorted_means=True, ascending=True, colored_patches=False, x_range_=[-0.1,1],
                 title='posterior traces', jd_mode=False, line_color=None, line_color_match=None, y_range_add=1)

# Use `vis_index` value to visualise prediction of a single datapoint

## e.g.
## plot_pdf(vis_index=INDEX HERE, x_range_=[-0.1, 1.2])

In [ ]:
plot_pdf(vis_index= 180, x_range_=[-0.1, 1.2])

In [ ]:
plot_pdf(vis_index= 37, x_range_=[-0.1, 1.2])

In [ ]:
plot_pdf(vis_index= 1347, x_range_=[-0.1, 1.2])